In [1]:
from random import shuffle
import glob
import csv
import sys
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [3]:
def loadImage(addr):
    img = cv2.imread(addr)
    if not img is None:
        # Convert it to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

In [4]:
def createDataRecord(outputAddr, inputAddrs, doAugs):
    writer = tf.io.TFRecordWriter(outputAddr)
    
    for i in range(len(inputAddrs)):
        
        if i % 1000 == 0:
            print("{0} of {1} images completed".format(i, len(inputAddrs)))
        
        imgNeutral = loadImage(inputAddrs[i][0])
        imgExpressive = loadImage(inputAddrs[i][1])
        
        if imgNeutral is None or imgExpressive is None:
            continue
        
        # This is an augmented sample, alter the expressive face
        if doAugs[i] == 1:
            imgExpressive = augment(imgExpressive)
        
        # Resize
        imgNeutral = cv2.resize(imgNeutral, (224, 224), interpolation=cv2.INTER_CUBIC)
        imgExpressive = cv2.resize(imgExpressive, (224, 224), interpolation=cv2.INTER_CUBIC)
        
        # Serialize
        feature = {
            'image_neutral': _bytes_feature(imgNeutral.tostring()),
            'image_expressive': _bytes_feature(imgExpressive.tostring())
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()



In [5]:
def rotateImage(image, angle):
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [6]:
def augment(x: tf.Tensor) -> tf.Tensor:
    whichAug = tf.random.uniform(shape=[1], minval=0, maxval=2, dtype=tf.dtypes.int32)
    if whichAug == 0:
        # Rotate
        rotation = tf.random.uniform(shape=[1], minval=-5, maxval=5, dtype=tf.dtypes.float32)
        x = rotateImage(x, rotation * math.pi / 180)
    elif whichAug == 1:
        # Translate (no more than 15 up or down)
        vertTrans = tf.random.uniform(shape=[1], minval=-9, maxval=10, dtype=tf.dtypes.int32)
        x = x[9+int(vertTrans):246+int(vertTrans), 9:246]
        
    sys.stdout.flush()
    return x

In [7]:
dbBasePath = 'C:\\Users\\Will\\Documents\\StoicNetData\\'

with open(dbBasePath + r'pairs.txt', newline = '') as pairData:
    pairReader = csv.reader(pairData, delimiter='\t')
    pairs = [[dbBasePath + p for p in pair] for pair in pairReader]

# Double the pairs, and label the second half to be altered for augmentation
augs = ([0] * len(pairs)) + ([1] * len(pairs))
pairs = pairs + list.copy(pairs)

# Shuffle the pairs (and keep the augmentation labels with them)
c = list(zip(pairs, augs))
shuffle(c)
pairs, augs = zip(*c)

# 60/20/20 train/val/test split
trainPairs = pairs[0:int(0.6*len(pairs))]
trainAugs = augs[0:int(0.6*len(augs))]
valPairs = pairs[int(0.6*len(pairs)):int(0.8*len(pairs))]
valAugs = augs[int(0.6*len(pairs)):int(0.8*len(augs))]
testPairs = pairs[int(0.8*len(pairs)):]
testAugs = augs[int(0.8*len(augs)):]


In [8]:
createDataRecord(dbBasePath + 'train.tfrecords', trainPairs, trainAugs)
createDataRecord(dbBasePath + 'val.tfrecords', valPairs, valAugs)
createDataRecord(dbBasePath + 'test.tfrecords', testPairs, testAugs)

0 of 12849 images completed
1000 of 12849 images completed
2000 of 12849 images completed
3000 of 12849 images completed
4000 of 12849 images completed
5000 of 12849 images completed
6000 of 12849 images completed
7000 of 12849 images completed
8000 of 12849 images completed
9000 of 12849 images completed
10000 of 12849 images completed
11000 of 12849 images completed
12000 of 12849 images completed
0 of 4283 images completed
1000 of 4283 images completed
2000 of 4283 images completed
3000 of 4283 images completed
4000 of 4283 images completed
0 of 4284 images completed
1000 of 4284 images completed
2000 of 4284 images completed
3000 of 4284 images completed
4000 of 4284 images completed
